# Assignment 2 - Machine learning and modeling
In this assignment, you work in the same groups you already are divided into. You need to define at least one task based on each of the exercises 4, 5 and 10. All together three tasks. You have to apply the tasks on the same dataset that you used in Assignment 1. 

You will hand in the assignment through a Jupyter notebook, along with your environment and the dataset you picked, both zipped together and submitted as one file. Please name your file so that it contains your group number. It is important that you clearly state the tasks you are performing on the dataset as questions or something similar in the notebook before you do the operations on the data. Also make sure to document your solutions and your thinking so that it can easily be followed. If you fail to do these things, you may not pass this assignment.

The deadline of this assignment is on April 26, 2025 to get bonus points, or before the exam (in which case no bonus points will be awarded).

Re-submission 1 is by the end of week 33, 2025.

Re-submission 2 is by the end of week 2, 2026.

# Exercice 4
The tasks are
- Regression
- Clustering
- Decision Trees and Model validation
- SVMs, Hyperparameters, and Cross-Validation

# Exercice 5
The tasks are:
- Permutation feature importance
- Statistical testing
- Dimensionality reduction


## Import and dataset

In [ ]:
try:
    from rich import load_ipython_extension
    %load_ext rich
except ImportError:
    try:
        from rich import pretty
        pretty.install()
    except ImportError:
        pass

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
df = pd.read_csv("../Total air emissions by greenhouse gas.csv")
df

## Regression

In [ ]:
# get x axis, column
col = df.columns
col = col[2:]
x = [int(x) for x in col]
print(x)

# get y axis, first row value
first_row = df.iloc[0].tolist()
first_row = first_row[2:]
y = [float(x) for x in first_row]
print(y)

plt.scatter(x, y)
plt.show()

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

x = np.array(x).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)

model = LinearRegression(fit_intercept=True)
model.fit(x, y)

print("Coefficient:", model.coef_)
print("Intercept:", model.intercept_)

yfit = model.predict(x)
plt.scatter(x, y)
plt.plot(x, yfit, color='orange', lw=2)
plt.show()

### Polynomial regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Polynomial 2 cause i don't think more is needed
poly_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
poly_model.fit(x, y)

yfit = poly_model.predict(x)

plt.scatter(x, y)
plt.plot(x, yfit, color='orange', lw=2)


## Clustering


In [ ]:
# get more rows, the first 4
def get_row(i):
    row = df.iloc[i].tolist()
    row = row[2:]                       # remove first 2 col
    convert = [float(x) for x in row]   # convert to float
    return np.array(convert).reshape(-1, 1)

y_0 = get_row(0)
y_1 = get_row(1)
y_2 = get_row(2)
y_3 = get_row(3)

plt.scatter(x, y_0, color='red')
plt.scatter(x, y_1, color='blue')
plt.scatter(x, y_2, color='green')
plt.scatter(x, y_3, color='purple')
plt.show()

### K Mean

In [ ]:
X = []
Y = []
for index, y in enumerate([y_0, y_1, y_2, y_3]):
    X.append(np.column_stack((x, y)))
    Y.extend([index for _ in range(len(y))])

X = np.concatenate(X, axis=0)
Y = np.array(Y)
print(X)
print(Y)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(4, random_state=1)
labels = kmeans.fit(X).predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis')

### Gaussian Mixture Model (GMM)

In [ ]:
from sklearn.mixture import GaussianMixture as GMM

gmm = GMM(n_components=4, random_state=0)
labels = gmm.fit(X).predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis')

### Run GMM several times

In [ ]:
gmm = GMM(n_components=4, n_init=10)
labels = gmm.fit(X).predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis')

As we can see it's not very good to get cluster from time series

## Decision tree
### Split the data

In [ ]:
from sklearn.model_selection import train_test_split

print(X)
print(Y)
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, train_size=0.8
)

### Create and train the decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

y_pred = tree.predict(X_test)
y_pred

### Evaluate the decision tree performance

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

print("Absolute error:", (y_test != y_pred).sum())
print("Accuracy:", accuracy_score(y_test, y_pred))

print("F1-scores (macro):", f1_score(y_test, y_pred, average='macro'))
print("F1-scores (weighted):", f1_score(y_test, y_pred, average='weighted'))

### Plot a confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')

### Plot the decision tree

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(10, 6))
plot_tree(tree)

it's bigger than i thought so why not reduce it and retest it ? maybe it will be better to generalize.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Change the max depth to 4, we see on the figure above the depth is 6 with no pre-pruning
tree = DecisionTreeClassifier(random_state=0, max_depth=4).fit(X_train, y_train)

y_pred = tree.predict(X_test)
from sklearn.metrics import f1_score, accuracy_score

print("Absolute error:", (y_test != y_pred).sum())
print("Accuracy:", accuracy_score(y_test, y_pred))

print("F1-scores (macro):", f1_score(y_test, y_pred, average='macro'))
print("F1-scores (weighted):", f1_score(y_test, y_pred, average='weighted'))

plt.figure(figsize=(10, 6))
plot_tree(tree)

Nope, reducing the max depth doesn't improve the performance in our case.

# Exercice 5 
## Permutation feature importance

### importing again, to be clear 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

df = pd.read_csv("../Total air emissions by greenhouse gas.csv")

### cleaning data, remplacing NaN per the mean for each columns, for more data

In [ ]:

years = [str(year) for year in range(1990, 2024)]
for year in years:
    df[year] = pd.to_numeric(df[year], errors='coerce')

df[years] = df[years].fillna(df[years].mean())

### created a new target : did the consumption grow or not between 2023 and 1990 ? 

In [ ]:
df['growth'] = df['2023'] - df['1990']
df['target'] = (df['growth'] > 0).astype(int)  # 1 is true, 0 is false

X = df[years]
y = df['target']

print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

### Splitting, creating the model for trying to predict the growth, training, getting the accuracy score and the importance 

In [ ]:

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# Modèle
model = SVC(C=1.0, gamma=0.0001, kernel="rbf")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Score
print(f"Accuracy : {accuracy_score(y_pred, y_test):.4f}")

# Permutation importance
pi = permutation_importance(model, X_test, y_test, n_repeats=50, random_state=0)
importances = pd.DataFrame({'feature': X.columns, 'importance': pi['importances_mean']})
importances = importances.sort_values(by='importance', ascending=False)
print(importances)

### most important features

In [ ]:
# Get the most important feature
most_important_feature = importances.iloc[0]
print(f"The most important feature is: {most_important_feature['feature']} with importance: {most_important_feature['importance']}")

### train with the two most important features

In [ ]:
# Select the two most important features
top_2_features = importances['feature'].head(2).values
X_train_top_2 = X_train[top_2_features]
X_test_top_2 = X_test[top_2_features]

# Train the model using these two features
model.fit(X_train_top_2, y_train)

# Predict and evaluate accuracy
y_pred_top_2 = model.predict(X_test_top_2)
accuracy_top_2 = accuracy_score(y_pred_top_2, y_test)

print(f"Accuracy with the top 2 features: {accuracy_top_2:.4f}")


### two less important : 

In [ ]:
# two least important features
bottom_2_features = importances['feature'].tail(2).values
X_train_bottom_2 = X_train[bottom_2_features]
X_test_bottom_2 = X_test[bottom_2_features]

# Train the model using these
model.fit(X_train_bottom_2, y_train)

# Predict and evaluate accuracy
y_pred_bottom_2 = model.predict(X_test_bottom_2)
accuracy_bottom_2 = accuracy_score(y_pred_bottom_2, y_test)

print(f"Accuracy with the bottom 2 features: {accuracy_bottom_2:.4f}")


### as we can see, our top and bottom are not very reflective on the importance of them, as we got a better score for bottom rather than top features. we can ever do a cross validation to check :

In [ ]:
from sklearn.model_selection import cross_val_score

X_top_2 = X[top_2_features]
cv_scores_top_2 = cross_val_score(model, X_top_2, y, cv=5)
print(f"Cross-validation scores with top 2 features: {cv_scores_top_2}")
print(f"Mean CV score with top 2 features: {np.mean(cv_scores_top_2):.4f}")

X_bottom_2 = X[bottom_2_features]
cv_scores_bottom_2 = cross_val_score(model, X_bottom_2, y, cv=5)
print(f"Cross-validation scores with bottom 2 features: {cv_scores_bottom_2}")
print(f"Mean CV score with bottom 2 features: {np.mean(cv_scores_bottom_2):.4f}")


### this might be due to overfitting, or also because the bottom data have at the end better pattern than the top data. 

## Statistical testing

### thanks to statistical testing, we can prove, or unprove our result from above, saying that the top and bottom are not really relevant.

### re-doing step 2, for clarity.

In [ ]:
import pandas as pd
import numpy as np
import pingouin as pg
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score

# Load and clean dataset
df = pd.read_csv("../Total air emissions by greenhouse gas.csv")
years = [str(year) for year in range(1990, 2024)]
for year in years:
    df[year] = pd.to_numeric(df[year], errors='coerce')
df[years] = df[years].fillna(df[years].mean())

df['growth'] = df['2023'] - df['1990']
df['target'] = (df['growth'] > 0).astype(int)

X = df[years]
y = df['target']

# Split and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
model = SVC(C=1.0, gamma=0.0001, kernel="rbf")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Permutation importance
pi = permutation_importance(model, X_test, y_test, n_repeats=50, random_state=0)
importances = pd.DataFrame({'feature': X.columns, 'importance': pi['importances_mean']})
importances = importances.sort_values(by='importance', ascending=False)

# Select features
top_2_features = importances['feature'].head(2).values
bottom_2_features = importances['feature'].tail(2).values

# Re-train models with top 2 and bottom 2 features
model_top2 = SVC(C=1.0, gamma=0.0001, kernel="rbf")
model_bottom2 = SVC(C=1.0, gamma=0.0001, kernel="rbf")

### training again, for bottom2, top2, majority

In [ ]:


X_train_top2 = X_train[top_2_features]
X_test_top2 = X_test[top_2_features]
X_train_bottom2 = X_train[bottom_2_features]
X_test_bottom2 = X_test[bottom_2_features]

model_top2.fit(X_train_top2, y_train)
model_bottom2.fit(X_train_bottom2, y_train)

y_pred_top2 = model_top2.predict(X_test_top2)
y_pred_bottom2 = model_bottom2.predict(X_test_bottom2)

model_majority = DummyClassifier(strategy="most_frequent")
model_majority.fit(X_train_top2, y_train)
y_pred_majority = model_majority.predict(X_test_top2)

### building a classifier correctness dataframe

In [ ]:

df_eval = pd.DataFrame({
    'top2': y_pred_top2 == y_test.values,
    'bottom2': y_pred_bottom2 == y_test.values,
    'majority': y_pred_majority == y_test.values
})

### performed McNemar test bettwen the top 2 and the bottom 2 

In [ ]:

observed1, stats1 = pg.chi2_mcnemar(df_eval, x='bottom2', y='majority')
print("McNemar's test between bottom2 and majority classifier:")
print(stats1)

observed2, stats2 = pg.chi2_mcnemar(df_eval, x='top2', y='bottom2')
print("McNemar's test between top2 and bottom2 classifier:")
print(stats2)


### Checking normality on original features 

In [ ]:
df_sub = df[years]
normality = pg.normality(df_sub)
print("Normality test results:")
print(normality)


### bottom2 vs majority has a p of 0.4795, no real difference

### top2 vs bottom2 has a p of 1, so no difference at all

# Correlation analysis (Pearson, since normality generally ok on large samples)

In [ ]:
correlation = pg.pairwise_corr(df_sub, method="pearson")
correlation

# Dimensionality reduction

### performing pca and converting the pca output into a df with the original data index (optional)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)  # X : year 1990-2023 which are your features

# converting the PCA output into a df with the original data index (optional)
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['target'] = y  # add target to the DataFrame for color-coding in the plot


### Plot the 2D representation of the PCA-transformed data

In [ ]:

plt.figure(figsize=(10, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['target'], cmap='viridis')
plt.title('PCA - 2D Projection of the Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Target (Growth > 0)')
plt.show()

We can see outliers, some datapoint with unusual feature value. PC1 is capturing more variance than PC2. But the clusters are not very distincts. And PCA is not really helping

In [ ]:
from sklearn.manifold import TSNE

# Function to run t-SNE and plot the results
def run_tsne(X, y, random_state=0, perplexity=30):
    tsne = TSNE(n_components=2, random_state=random_state, perplexity=perplexity)
    X_tsne = tsne.fit_transform(X)

    tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
    tsne_df['target'] = y

    plt.figure(figsize=(10, 6))
    plt.scatter(tsne_df['TSNE1'], tsne_df['TSNE2'], c=tsne_df['target'], cmap='viridis')
    plt.title(f't-SNE with random_state={random_state}, perplexity={perplexity}')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.colorbar(label='Target (Growth > 0)')
    plt.show()

# Run t-SNE with default parameters
run_tsne(X, y)

# Experiment with different values of perplexity and random_state
for perplexity in [5, 10, 25, 50]:
    run_tsne(X, y, random_state=42, perplexity=perplexity)


### We can see some clustering patterns in these t-SNE plots, which add more to the data than PCA, the two color are more separated.